In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# -------------------------------------------------------
# 1) LOAD DATA
# -------------------------------------------------------
df = pd.read_csv("student_performance_dataset_20.csv")

# Select required features
X = df[['Study_Hours_per_Week', 'Attendance_Rate', 'Internal_Scores']].values
y = df['Pass_Fail'].map({'Pass': 1, 'Fail': -1}).values

# -------------------------------------------------------
# 2) SPLIT TRAIN & TEST
# -------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# -------------------------------------------------------
# 3) POLYNOMIAL KERNEL FUNCTION
# -------------------------------------------------------
def poly_kernel(x, z, degree=2):
    return (np.dot(x, z) + 1) ** degree

# -------------------------------------------------------
# 4) SVM FROM SCRATCH (DUAL FORM APPROX, PEGASOS STYLE)
# -------------------------------------------------------
def train_kernel_svm(X, y, lr=0.01, lambda_param=0.01, epochs=15, degree=2):
    n = X.shape[0]
    alpha = np.zeros(n)

    # Kernel matrix
    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i, j] = poly_kernel(X[i], X[j], degree)
    
    # Gradient-like updates on dual coefficients
    for _ in range(epochs):
        for i in range(n):
            condition = y[i] * np.sum(alpha * y * K[:, i]) < 1
            if condition:
                alpha[i] += lr
            else:
                alpha[i] -= lr * lambda_param
    return alpha, K

alpha, K_train = train_kernel_svm(X_train, y_train, degree=2)

# -------------------------------------------------------
# 5) PREDICTION FUNCTION
# -------------------------------------------------------
def predict_kernel_svm(X_train, X_test, y_train, alpha, degree=2):
    preds = []
    for x in X_test:
        score = 0
        for i in range(len(alpha)):
            score += alpha[i] * y_train[i] * poly_kernel(X_train[i], x, degree)
        preds.append(1 if score >= 0 else -1)
    return np.array(preds)

y_pred = predict_kernel_svm(X_train, X_test, y_train, alpha, degree=2)

# Convert to binary labels for evaluation
y_test_label = np.where(y_test == 1, 1, 0)
y_pred_label = np.where(y_pred == 1, 1, 0)

# -------------------------------------------------------
# 6) EVALUATION METRICS
# -------------------------------------------------------
precision = precision_score(y_test_label, y_pred_label)
recall = recall_score(y_test_label, y_pred_label)
f1 = f1_score(y_test_label, y_pred_label)
cm = confusion_matrix(y_test_label, y_pred_label)

print("\nConfusion Matrix:\n", cm)
print("\nPrecision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



Confusion Matrix:
 [[86 22]
 [13 92]]

Precision: 0.8070175438596491
Recall: 0.8761904761904762
F1 Score: 0.8401826484018264


In [3]:
def predict_student(study_hours, attendance_rate, internal_score):
    # Convert input to scaled vector
    x = np.array([[study_hours, attendance_rate, internal_score]])
    x = scaler.transform(x)   # scale same as training
    
    score = 0
    for i in range(len(alpha)):
        score += alpha[i] * y_train[i] * poly_kernel(X_train[i], x[0], degree=2)
    
    return "PASS" if score >= 0 else "FAIL"


In [4]:
print("Test Case 1:", predict_student(12, 90, 78))   # Studies well, attends regularly
print("Test Case 2:", predict_student(3, 65, 40))     # Low study, weak internal score
print("Test Case 3:", predict_student(8, 85, 55))     # Average performance
print("Test Case 4:", predict_student(1, 45, 20))     # Very poor engagement
print("Test Case 5:", predict_student(15, 95, 88))    # Top performer


Test Case 1: PASS
Test Case 2: FAIL
Test Case 3: FAIL
Test Case 4: FAIL
Test Case 5: PASS
